Import necessary files and libs:

In [2]:
import pandas as pd
import requests as r
from files.lol_regions import regions
from files.lol_genders import genders
from files.lol_names import correction_roles

##### __Champion gender:__

Create dataframe:

In [3]:
lol_df = pd.read_csv("files/data.csv")

Add gender to respective champions:

In [4]:
for k,v in genders.items():
    lol_df.loc[lol_df["champion"] == k, "gender"] = v

In [ ]:
lol_df

##### __Champion roles:__

Get list from API of all champions:

In [5]:
patch = "12.12.1"
ddragon = r.get(f"http://ddragon.leagueoflegends.com/cdn/{patch}/data/en_US/champion.json").json()
champions = list(ddragon['data'].keys())

Create and format list with champion roles:

In [6]:
list_tags = []
for c in champions:
    tag_champion = ddragon['data'][c]['tags']
    try:
        list_tags.append([c, f"{tag_champion[0]}, {tag_champion[1]}"])
    except:
        list_tags.append([c, f"{tag_champion[0]}"])

Create dataframe and apply name correction:

In [7]:
tags_df = pd.DataFrame(list_tags, columns=["champion", "role"])
for key, value in correction_roles.items():
    tags_df.loc[tags_df["champion"] == key, "champion"] = value

Merge dataframes:

In [8]:
lol_df = lol_df.merge(tags_df)

In [ ]:
lol_df